In [1]:
import tensorflow as tf
import numpy as np
import models
import helpers
import data_utils

In [2]:
metadata, idx_q, idx_a = data_utils.ourmodel.data_util.load_data()
(trainX, trainY), (testX, testY), (validX, validY) = helpers.split_dataset(idx_q, idx_a)

In [3]:
reload(data_utils.ourmodel.data_util)
metadata = data_utils.ourmodel.data_util.load_metadata()

In [4]:
len(metadata['idx2w']) 

174539

In [9]:
BATCH_SIZE = 64
def batch():
    X = np.load('datasets/idx_review.npy', mmap_mode='r')
    Y =  np.load('datasets/idx_summary.npy', mmap_mode='r')
    
    i = np.random.randint(len(X))
    print X.shape
    return X[i:i+BATCH_SIZE],Y[i:i+BATCH_SIZE]
for i in range(1):
    
    summary,review = batch()

(561232, 200)


In [10]:
for ii, oi in zip(summary, review):
    q = helpers.decode(sequence=ii, lookup=metadata['idx2w'], separator=' ')
    decoded = helpers.decode(sequence=oi, lookup=metadata['idx2w'], separator=' ').split(' ')
    print('Review : [{0}]; Summary : [{1}] '.format(q, ' '.join(decoded)))
    #print "Real Summary %s",(helpers.decode(sequence=ot, lookup=metadata['idx2w'], separator=' ').split(' '))


Review : [love bars sweet salty low calories dont much soy protein isolate taste good texture great definitely treat decent amount fiber recommended]; Summary : [really delicious] 
Review : [quite good 90 calories 77 ounces taste like chewy chocolate covered pretzels satisfying currently available amazon found walmart 250 per box 6]; Summary : [quite good hard find] 
Review : [ive already gone several boxes amazing little bars theyre unk perfect blend sweet unk cant believe theyre 90 calories excellent snack definitely worth buying]; Summary : [wow] 
Review : [item great watching calories need little something tide till lunch dinner sweetness salty flavor goes great together really hits spot]; Summary : [great tasting] 
Review : [things awesome price wonderful subscribe save best 2 point bar market weight watchers try bar savior love amazon]; Summary : [2 unk weight watchers] 
Review : [local grocery store introduced stopped carrying flavor go figure amazon came enough last good long t

In [5]:

# parameters 
xseq_len = trainX.shape[-1]
yseq_len = trainY.shape[-1]
batch_size = 16
xvocab_size = len(metadata['idx2w'])  
yvocab_size = xvocab_size
emb_dim = 64


In [ ]:
trainXX = trainX[1:1000]
trainYY = trainY[1:1000]

In [ ]:
reload(models.seq2seq_wrapper)
from models import seq2seq_wrapper
model = seq2seq_wrapper.Seq2Seq(xseq_len= 50,
                               yseq_len= 20,
                               xvocab_size=xvocab_size,
                               yvocab_size=yvocab_size,
                               ckpt_path='ckpt/checkpoint/',
                               emb_dim=emb_dim,
                               num_layers=2,
                             epochs = 10000,
                                lr = 0.005,
                                attention = True,
                                celltype = 'GRU'
                               )
sess = tf.Session()
            # init all variables
sess.run(tf.global_variables_initializer())

<log> Building Graph <log> Building Input variables of Graph <log> Cells 

In [10]:
batch_size = 100

val_batch_gen = helpers.rand_batch_gen(trainXX, trainYY, 16)
train_batch_gen = helpers.rand_batch_gen(trainXX, trainYY, batch_size)

In [ ]:
sess = model.train(train_batch_gen, val_batch_gen)

training started

Model saved to disk at iteration #10
val   loss : 1.777225

Model saved to disk at iteration #20
val   loss : 0.648790

Model saved to disk at iteration #30
val   loss : 0.612422

Model saved to disk at iteration #40
val   loss : 0.405445

Model saved to disk at iteration #50
val   loss : 0.457402

Model saved to disk at iteration #60
val   loss : 0.415419

Model saved to disk at iteration #70
val   loss : 0.421635

Model saved to disk at iteration #80
val   loss : 0.446304

Model saved to disk at iteration #90
val   loss : 0.394088

Model saved to disk at iteration #100
val   loss : 0.384027

Model saved to disk at iteration #110
val   loss : 0.406275

Model saved to disk at iteration #120
val   loss : 0.339208

Model saved to disk at iteration #130
val   loss : 0.354319

Model saved to disk at iteration #140
val   loss : 0.405343

Model saved to disk at iteration #150
val   loss : 0.365870

Model saved to disk at iteration #160
val   loss : 0.404561

Model saved to 

In [8]:

sess = model.restore_last_session()

In [11]:
input_,output_ = train_batch_gen.next()
output = model.predict(sess, input_)
print(output.shape)

(100, 30)


In [12]:
replies = []

for ii, oi,ot in zip(input_.T, output,output_.T):
    q = helpers.decode(sequence=ii, lookup=metadata['idx2w'], separator=' ')
    decoded = helpers.decode(sequence=oi, lookup=metadata['idx2w'], separator=' ').split(' ')
    rs = helpers.decode(sequence=ot, lookup=metadata['idx2w'], separator=' ').split(' ')
    #if decoded.count('unk') == 0:
     #   if decoded not in replies:
    print('Review : [{0}]; Summary : [{1}];real summary :[{2}] '.format(q, ' '.join(decoded),' '.join(rs)))
    #print "Real Summary %s",(helpers.decode(sequence=ot, lookup=metadata['idx2w'], separator=' ').split(' '))
    replies.append(decoded)

Review : [tried chips hotel stayed loved ordered office everyone enjoyed im second order]; Summary : [great good chips];real summary :[yummy healthy snack] 
Review : [peanut lover much larger cocktail peanuts six people christmas gift list ask every year]; Summary : [great];real summary :[best] 
Review : [dog loves loves dog unk say love twice dog food one dogs favorite]; Summary : [great good chips];real summary :[dog loves dog food] 
Review : [office personally smoothest best tasting coffee buy keurig machine great price]; Summary : [great good];real summary :[best overall coffee] 
Review : [popchips good following weight watchers lost 50 pounds goal popchips salty crunchy indulgence taste good]; Summary : [great good chips];real summary :[love love love] 
Review : [second order good hot like best sample per preferences]; Summary : [great good chips];real summary :[curry paste] 
Review : [makes feeding go easy unk suck pouch 5 minutes loves]; Summary : [great good];real summary :[awe

In [ ]:
input_,output_ = val_batch_gen.next()
output = model.predict(sess, input_)
print(output.shape)

In [ ]:
import sys
from nltk.corpus import stopwords
sys.stdout.write("> ")
sys.stdout.flush()
sentence = "This is great product"

In [ ]:
inputs = data_utils.ourmodel.data_util.get_tokens(sentence)
fqtokens =  [w for w in inputs if not w in stopwords.words('english')]

In [ ]:
reload(data_utils.ourmodel.data_util)
from data_utils.ourmodel.data_util import zero_pad_single
processed_input = zero_pad_single(fqtokens,metadata['w2idx'])

In [ ]:
processed_input.shape